# Initial experiments with embeddings model

In [1]:
import sys
sys.path.insert(0, '../train')

### Download from hub (or load cached file)

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from pathlib import Path

SAVED_MODEL_DIR = Path('../data/models/saved_speech_embedding_model')

if SAVED_MODEL_DIR.exists():
    loaded_model = tf.saved_model.load(SAVED_MODEL_DIR)
else:
    loaded_model = hub.load('https://tfhub.dev/google/speech_embedding/1')
    tf.saved_model.save(loaded_model, SAVED_MODEL_DIR, signatures=loaded_model.signatures)

loaded_model_default = loaded_model.signatures["default"]

print('Signatures:', list(loaded_model.signatures.keys()))

2025-07-31 21:08:35.827011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753988915.851420   18174 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753988915.856453   18174 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753988915.872315   18174 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753988915.872336   18174 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753988915.872338   18174 computation_placer.cc:177] computation placer alr

Signatures: ['default']


### Convert to tflite

In [3]:
TFLITE_MODEL_FILE = Path('../data/models/embedding_model.tflite')

if not TFLITE_MODEL_FILE.exists():
    converter = tf.lite.TFLiteConverter.from_concrete_functions([loaded_model_default])
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    TFLITE_MODEL_FILE.write_bytes(converter.convert())

print(f'\nYou can inspect {TFLITE_MODEL_FILE.resolve()} with https://netron.app/')


You can inspect /home/doki/my/home-asist/wakeupword/data/models/embedding_model.tflite with https://netron.app/


### Inspect in tensorboard

In [4]:
writer = tf.summary.create_file_writer("../logdir")
with writer.as_default():
    tf.summary.graph(loaded_model_default.graph)
writer.close()
print ('RUN IN TERMINAL:\n', 'tensorboard --logdir=./logdir')

RUN IN TERMINAL:
 tensorboard --logdir=./logdir


### Detect windowing and sliding sizes

In [5]:
def binary_search(start, end, callback):
    low = start
    high = end
    while low < high:
        mid = (low + high) // 2
        if callback(mid):
            high = mid
        else:
            low = mid + 1
    return low if low < end else None

min_samples = binary_search(12000, 200000, lambda x: int(tf.size(loaded_model_default(tf.random.uniform(shape=(1, x), dtype=tf.float32))['default'])) > 0)
after_step1_samples = binary_search(min_samples, 200000, lambda x: int(tf.size(loaded_model_default(tf.random.uniform(shape=(1, x), dtype=tf.float32))['default'])) > 96)
after_step2_samples = binary_search(min_samples, 200000, lambda x: int(tf.size(loaded_model_default(tf.random.uniform(shape=(1, x), dtype=tf.float32))['default'])) > 2 * 96)
step_samples = after_step1_samples - min_samples
print(f'Minimum samples: {min_samples} samples = {min_samples / 16} ms')
print(f'Sliding step: {step_samples} samples = {step_samples / 16} ms')
print(f'Sliding step: {after_step2_samples - after_step1_samples} samples = {(after_step2_samples - after_step1_samples) / 16} ms')

Minimum samples: 12400 samples = 775.0 ms
Sliding step: 1280 samples = 80.0 ms
Sliding step: 1280 samples = 80.0 ms


### Compare my mel spectrogram with model

In [18]:
import numpy as np
from mel import calc_mel

data = np.random.uniform(size=(min_samples), low=-1.0, high=1.0).astype(np.float32)
my_mel = calc_mel(data)

model_tflite = tf.lite.Interpreter(model_path=str(TFLITE_MODEL_FILE), experimental_preserve_all_tensors=True)
model_tflite.resize_tensor_input(
    model_tflite.get_input_details()[0]['index'],
    [1, min_samples],
    strict=True)
model_tflite.allocate_tensors()
model_tflite_in = model_tflite.get_input_details()[0]['index']
model_tflite_out = model_tflite.get_output_details()[0]['index']
model_tflite.set_tensor(model_tflite_in, data.reshape(1, -1))
model_tflite.invoke()

tensor_details = model_tflite.get_tensor_details()

for d in tensor_details:
    if d['name'] == 'Squeeze':
        model_mel = model_tflite.get_tensor(d['index'])
        break
else:
    raise ValueError('"Squeeze" tensor not found in TFLite model')

print(f'Mel shape (my): {my_mel.shape}')
print(f'Mel shape (TFLite): {model_mel.shape}')

diff = my_mel - model_mel.reshape(my_mel.shape)
print(f'Max diff: {np.max(np.abs(diff))}')


Mel shape (my): (76, 32)
Mel shape (TFLite): (1, 76, 32)
Max diff: 6.556510925292969e-07
